In [1]:
import pandas as pd
from pathlib import Path
from hict.api.ContactMatrixFacet import ContactMatrixFacet
from hict.util.matrix_visualise import MatrixVisualise
import cooler
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from keras.models import load_model
import glob

# Setup constants and load models

In [2]:
img_height = 20
img_width = 20

centered_first = load_model("D:/Study/HICT/HICT_Patterns/exported_models/centered_first.keras")
centered_second = load_model("D:/Study/HICT/HICT_Patterns/exported_models/centered_second.keras")
centered_third = load_model("D:/Study/HICT/HICT_Patterns/exported_models/centered_third.keras")

final_first = load_model("D:/Study/HICT/HICT_Patterns/exported_models/final_first.keras")
final_second = load_model("D:/Study/HICT/HICT_Patterns/exported_models/final_second.keras")
final_third = load_model("D:/Study/HICT/HICT_Patterns/exported_models/final_third.keras")

# Create empty directories for temp files

In [ ]:
def delete_temp_images(path):
    files = glob.glob(f'{path}/*')
    for f in files:
        os.remove(f)

chr = 'scf5'
resolution = 50000
dataset_path = f'datasets/temp_images_{resolution//1000}'
unknown_images_path = f'datasets/temp_images_{resolution//1000}/unknown'

if not os.path.exists(dataset_path):
    os.mkdir(dataset_path)

if not os.path.exists(unknown_images_path):
    os.mkdir(unknown_images_path)
delete_temp_images(unknown_images_path)

# Load matrix from source

In [ ]:
#c = cooler.Cooler(f'data/atrop_ebre/atrop_ebre_4DN.mcool::/resolutions/{resolution}')
#matrix = c.matrix(balance=False).fetch(chr)

chunkFile = ContactMatrixFacet.get_file_descriptor('data/arab_dongX_vs_mopti_2nd_try.hdf5')
ContactMatrixFacet.open_file(chunkFile)
ContactMatrixFacet.load_assembly_from_agp(chunkFile, Path('data/mopti_female_assembly_2.agp'))
matrix = MatrixVisualise.get_matrix(chunkFile, 'scaffold_5', resolution, False)

# Create samples for detection from matrix main diagonal

In [ ]:
src_matrix = np.log10(matrix+1)
max_value = np.nanmax(src_matrix)
min_value = np.nanmin(src_matrix)
for i in range(10, src_matrix.shape[0]-10):
        point = (i, i)
        point_area = src_matrix[point[1]-10:point[1]+10, point[0]-10:point[0]+10]
        plt.imsave(f'{unknown_images_path}/{chr}_{i*resolution}_{i*resolution}.png',  point_area, cmap='gray', vmax=max_value, vmin=min_value)

# Primary detection by samples from main diagonal only

In [ ]:
detected_points = []

for path in os.listdir(unknown_images_path):
    img_path = os.path.join(unknown_images_path, path)
    with tf.device('/device:CPU:0'):
        img = tf.keras.utils.load_img(
            img_path, target_size=(img_height, img_width)
        )
        img_array = tf.keras.utils.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create a batch

        predictions_cnn = np.array(centered_first(img_array)),
        predictions_cnn_2 = np.array(centered_second(img_array)),
        predictions_LeNet = np.array(centered_third(img_array)),
        predictions = (predictions_cnn[0] + predictions_cnn_2[0] + predictions_LeNet[0]) / 3

        score = tf.nn.softmax(predictions[0])

        if np.argmax(score) == 1:
          path_splited = path.split('_')
          detected_points.append(int(path_splited[1]))

# Create samples from crosses of detected translocations on main diagonal

In [ ]:
detected_images_path = f'{dataset_path}/detected'
if not os.path.exists(detected_images_path):
    os.mkdir(detected_images_path)
delete_temp_images(detected_images_path)

for i in detected_points:
    for j in detected_points:
        if i == j or i > j:
            continue
        point = (i//resolution, j//resolution)
        point_area = src_matrix[point[1]-10:point[1]+10, point[0]-10:point[0]+10]
        plt.imsave(f'{detected_images_path}/{chr}_{i}_{j}.png',  point_area, cmap='gray', vmax=max_value, vmin=min_value)

# Final detection by samples from whole matrix

In [ ]:
with open('data/ab_detected_rearrangements_ensemble_crossed_10.csv', mode='w') as output:
    output.write("chr,start,end\n")
    for path in os.listdir(detected_images_path):
        img_path = os.path.join(detected_images_path, path)
        with tf.device('/device:CPU:0'):
            img = tf.keras.utils.load_img(
                img_path, target_size=(img_height, img_width)
            )
            img_array = tf.keras.utils.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0) # Create a batch

            predictions_cnn = np.array(final_first(img_array)),
            predictions_cnn_2 = np.array(final_second(img_array)),
            predictions_LeNet = np.array(final_third(img_array)),
            predictions = (predictions_cnn[0] + predictions_cnn_2[0] + predictions_LeNet[0]) / 3

            score = tf.nn.softmax(predictions[0])

            if np.argmax(score) == 1:
              path_splited = path.split('_')
              output.write(f"X,{path_splited[1]},{path_splited[2].split('.')[0]}\n")

# Visualising results

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
im = ax.matshow(np.log10(matrix+1), cmap='Greens')
fig.colorbar(im)

trans = pd.read_csv('data/ab_detected_rearrangements_ensemble_crossed_10.csv')
trans['start_res'] = trans.start // resolution
trans['end_res'] = trans.end // resolution
plt.scatter(trans.start_res, trans.end_res,s=0.1, c='#E9967A')
plt.savefig('ab_trans_center_unbalanced_crossed_10k.png', dpi=1000)